In [1]:
import requests
import zipfile
import time
import os
import unicodedata
from urllib.parse import urljoin
from io import BytesIO
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

In [2]:
sslverify = True  # nastavit na False u serveru, ktere hlasi chybu ssl
urlRoot = "https://volby.cz/opendata/"
opendatafile = "opendata.htm"
datadir = "data"
opendataUrl = urljoin(urlRoot,opendatafile)


In [3]:
response = requests.get(opendataUrl)

In [4]:
def getUrlsVolby(soup):
    volbyList = []
    for volby in soup.table.find_all("tr"):
        volbyData = volby.find("td").text
        volbyData = unicodedata.normalize('NFKD',volbyData)
        #print(volbyData)
        #print(volba)
        volbyUrls = volby.find_all("a")
        for i in volbyUrls:
            volba = []
            volba.append(volbyData)
            url = i.attrs["href"]
            # print(url)
            volbyId, volbyFileurl = url.split("/")
            volba.append(volbyId)
            volba.append(volbyFileurl)
            volbyList.append(volba)
            # print(volba)
            # print()
    return(volbyList)

In [5]:
if response.status_code == 200:
    HomeSoup = BeautifulSoup(response.content, 'html.parser')
    volbyUrls = getUrlsVolby(HomeSoup)
else:
    print("chyba zpracování", opendataUrl)

In [6]:
def zpracujOdata(url):
    odkazy = []
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for i in soup.find_all("li"):
        for a in i.find_all("a"):
            odkaz = a.attrs["href"]
            if "http" not in odkaz:
                if "_opendata.htm" in odkaz:
                    odkaz = odkaz.replace("..","/opendata")
                    odkaz = urljoin("https://volby.cz", odkaz)
                    odkazy.append(odkaz)
    return(odkazy)


volbaOpendataHome = []
for i in volbyUrls:
    volbaUrl = urlRoot + i[1] + "/" + i[2]
    time.sleep(0.3)
    if "odata" in volbaUrl or "_seznam" in volbaUrl:
        odkazy = zpracujOdata(volbaUrl)
        for odkaz in odkazy:
            volbaOpendataHome.append(odkaz)
    else:
        volbaOpendataHome.append(volbaUrl)

In [8]:
def getVolbySoup(home, sleeptime):
    responses = {}
    for i in home:
        volby = i.split("/")[4]
        responses[volby] = requests.get(i)
        time.sleep(sleeptime)
    return (responses)

In [11]:
volbySoupsDict = getVolbySoup(volbaOpendataHome, 2)

In [13]:
def getDavky(davkydir, url):
    posledni_davka = requests.get(url)
    print("getDavky", davkydir, url)
    root = ET.fromstring(posledni_davka.text)
    davka = root[0].attrib
    if 'CHYBA' in (root[0].tag):
        print(davkydir, root[0].attrib)
        return ()
    davkyLimit = int(davka.get("PORADI_DAVKY")) + 1
    print(davkydir, "\t", davkyLimit)
    if not os.path.exists(davkydir):
        os.mkdir(davkydir)
        for i in range(1, davkyLimit):
            response = requests.get(url, verify=sslverify)
            time.sleep(1)
            print("davka", i, response.status_code, "\r")
            time.sleep(0)  # workaround pro chybu v jupyteru - netiskne se \r

            if i < 10:
                davkaurl = os.path.join(davkydir, "davka_0"+str(i)+".xml")
            else:
                davkaurl = os.path.join(davkydir, "davka_"+str(i)+".xml")
            with open(davkaurl, "wb") as file:
                file.write(response.content)


def getZipDavky(volbadir, volbyDict):
    for volba, response in volbyDict.items():
        print(volba)
        volba = os.path.join(volbadir, volba)
        if not os.path.exists(volba):
            os.mkdir(volba)
        url = response.url
        soup = BeautifulSoup(response.content, 'html.parser')
        for a in soup.find_all('a'):
            odkaz = a['href']
            if '.zip' in odkaz:
                continue  # docasne zablokovat download zip
                fileUrl = urljoin(url, odkaz)
                dataType = fileUrl.split('/')[4]
                path = os.path.join(volba,dataType)
                r = requests.get(fileUrl)
                if r.status_code == 200:
                    z = zipfile.ZipFile(BytesIO(r.content))
                    z.extractall(path)
                time.sleep(1)
            elif 'davka=' in odkaz:
                fileUrl = urljoin(url, odkaz)
                print("fileUrl", fileUrl)
                dataType = fileUrl.split('/')[4]
                if ('se' in volba) or ('prez' in volba):
                    for i in range(1,3):
                        if 'kolo=' in fileUrl:
                            davkaUrl = fileUrl.replace("kolo=","kolo="+str(i))
                        else:
                            davkaUrl = fileUrl+"&kolo="+str(i)
                        path = os.path.join(volba, "davky_" + str(i))
                        getDavky(path, davkaUrl)
                else:
                    #print(fileUrl)
                    path = os.path.join(volba, "davky_1")
                    getDavky(path, davkaUrl)
            else:
                continue
        time.sleep(0.1)

In [ ]:
if not os.path.exists(datadir):
    os.mkdir(datadir)

getZipDavky(datadir, volbySoupsDict)